In [1]:
import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
sitemap = 'https://www.sfgate.com/sitemap.xml'
coverpage = requests.get(sitemap).content
data = xmltodict.parse(coverpage)
pages = data['sitemapindex']['sitemap']
results = [i['loc'] for i in pages]
df = pd.DataFrame(results, columns = ['page'])
df

,page
0,https://www.sfgate.com/sitemap/5000-10000.xml
1,https://www.sfgate.com/sitemap/55000-60000.xml
2,https://www.sfgate.com/sitemap/100000-105000.xml
3,https://www.sfgate.com/sitemap/115000-120000.xml
4,https://www.sfgate.com/sitemap/130000-135000.xml
...,...
2560,https://www.sfgate.com/sitemap/16105000-16110000.xml
2561,https://www.sfgate.com/sitemap/16110000-16115000.xml
2562,https://www.sfgate.com/sitemap/16115000-16120000.xml
2563,https://www.sfgate.com/sitemap/16120000-16125000.xml


In [15]:
headers={
    'accept': '*/*',
#     'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

def get_articles(page):
    try:
        coverpage = requests.get(page,headers = headers).content
        time.sleep(1)
        data = xmltodict.parse(coverpage)
        articles = data['urlset']['url']
        return [str(i) for i in articles]
    except:
        return

In [16]:
pandarallel.initialize(nb_workers= 64, progress_bar=True)
df['articles']= None
df['articles']= df['page'].parallel_apply(get_articles)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [21]:
df.to_csv('sfgate_1.csv', index=False)

In [2]:
df= pd.read_csv('sfgate_1.csv', header=0)
df

,page,articles
0,https://www.sfgate.com/sitemap/5000-10000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/'), ('lastmod', '2021-04-20T22:15:07Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/realestate/'), ('lastmod', '2021-04-14T20:12:18Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/sports/'), ('lastmod', '2021-04-06T15:50:21Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/rss/'), ('lastmod', '2021-03-31T16:57:41Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/raiders/'), ('lastmod', '2021-03-31T16:57:41Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/entertainment/'), ('lastmod', '2021-03-31T16:57:42Z')])"", ""OrderedDic..."
1,https://www.sfgate.com/sitemap/55000-60000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/cars/slideshow/2014-Lexus-IS-350-F-Sport-55002.php'), ('lastmod', '2013-01-11T20:43:52Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https://m.sfgate.com/cars/slideshow/2014-Lexus-IS-350-F-Sport-55002.php')]))])"", ""OrderedDict([('loc', 'https://www.sfgate.com/cars/slideshow/2014-Chevrolet-Silverado-55004.php'), ('lastmod', '2013-01-11T20:27:41Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https://m.sfgate...."
2,https://www.sfgate.com/sitemap/100000-105000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/business/slideshow/18-apps-that-earn-you-money-102106.php'), ('lastmod', '2018-08-17T13:13:02Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https://m.sfgate.com/business/slideshow/18-apps-that-earn-you-money-102106.php')]))])"", ""OrderedDict([('loc', 'https://www.sfgate.com/business/slideshow/Google-failures-101541.php'), ('lastmod', '2018-05-21T00:13:27Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https:/..."
3,https://www.sfgate.com/sitemap/115000-120000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/dearabby/slideshow/Dear-Abby-Best-of-Slideshow-118661.php'), ('lastmod', '2020-07-01T16:34:58Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https://m.sfgate.com/dearabby/slideshow/Dear-Abby-Best-of-Slideshow-118661.php')]))])"", ""OrderedDict([('loc', 'https://www.sfgate.com/news/slideshow/Historic-photos-of-Yosemite-National-Park-115230.php'), ('lastmod', '2020-01-07T20:27:00Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)..."
4,https://www.sfgate.com/sitemap/130000-135000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/news/slideshow/Best-Fall-Foliage-in-Bay-Area-and-Northern-134933.php'), ('lastmod', '2020-09-24T23:42:22Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only screen and (max-width: 640px)'), ('@href', 'https://m.sfgate.com/news/slideshow/Best-Fall-Foliage-in-Bay-Area-and-Northern-134933.php')]))])"", ""OrderedDict([('loc', 'https://www.sfgate.com/bayarea/slideshow/Masonic-temples-secret-meetinghouses-and-other-132855.php'), ('lastmod', '2020-03-21T16:24:38Z'), ('xhtml:link', OrderedDict([('@rel', 'alternate'), ('@media', 'only..."
...,...,...
2559,https://www.sfgate.com/sitemap/16105000-16110000.xml,"[""OrderedDict([('loc', 'https://www.sfgate.com/horoscope/article/Horoscope-for-Saturday-4-24-21-by-Christopher-16109103.php'), ('lastmod', '2021-04-24T05:05:52Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/horoscope/article/Horoscope-for-Friday-4-23-21-by-Christopher-16109091.php'), ('lastmod', '2021-04-23T05:05:52Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/horoscope/article/Horoscope-for-Thursday-4-22-21-by-Christopher-16109088.php'), ('lastmod', '2021-04-22T05:05:49Z')])"", ""OrderedDict([('loc', 'https://www.sfgate.com/renotahoe/article/camping-reservations-lake-tahoe-161..."
2560,https://www.sfga

In [4]:
df = df[~df.articles.isnull()].reset_index(drop=True)

In [5]:
df.articles = df.articles.parallel_apply(eval)
df = df.explode('articles').reset_index(drop=True)
df

,page,articles
0,https://www.sfgate.com/sitemap/5000-10000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/'), ('lastmod', '2021-04-20T22:15:07Z')])"
1,https://www.sfgate.com/sitemap/5000-10000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/realestate/'), ('lastmod', '2021-04-14T20:12:18Z')])"
2,https://www.sfgate.com/sitemap/5000-10000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/sports/'), ('lastmod', '2021-04-06T15:50:21Z')])"
3,https://www.sfgate.com/sitemap/5000-10000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/rss/'), ('lastmod', '2021-03-31T16:57:41Z')])"
4,https://www.sfgate.com/sitemap/5000-10000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/raiders/'), ('lastmod', '2021-03-31T16:57:41Z')])"
...,...,...
1101246,https://www.sfgate.com/sitemap/16120000-16125000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php'), ('lastmod', '2021-04-22T15:45:03Z')])"
1101247,https://www.sfgate.com/sitemap/16125000-16130000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/renotahoe/article/2021-04-earthquake-tahoe-truckee-reno-16127479.php'), ('lastmod', '2021-04-25T16:03:24Z')])"
1101248,https://www.sfgate.com/sitemap/16125000-16130000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/california-wildfires/article/2021-04-supertanker-plane-shut-down-calif-fires-16127394.php'), ('lastmod', '2021-04-25T14:42:28Z')])"
1101249,https://www.sfgate.com/sitemap/16125000-16130000.xml,"OrderedDict([('loc', 'https://www.sfgate.com/bayarea/article/2021-04-Fifth-whale-washes-up-dead-in-SF-Bay-16126769.php'), ('lastmod', '2021-04-25T01:44:09Z')])"


In [8]:
df = df[df.articles.str.startswith('Ordered')].reset_index(drop=True)

In [9]:
import collections
from collections import OrderedDict
import ast

# def convert(line):
#     values = re.search(r"OrderedDict\((.*)\)", line).group(1)
#     return dict(eval(values))

In [10]:
df['articles'] = df['articles'].parallel_apply(lambda x: re.search(r"OrderedDict\((.*)\)", x).group(1))
df

,page,articles
0,https://www.sfgate.com/sitemap/5000-10000.xml,"[('loc', 'https://www.sfgate.com/'), ('lastmod', '2021-04-20T22:15:07Z')]"
1,https://www.sfgate.com/sitemap/5000-10000.xml,"[('loc', 'https://www.sfgate.com/realestate/'), ('lastmod', '2021-04-14T20:12:18Z')]"
2,https://www.sfgate.com/sitemap/5000-10000.xml,"[('loc', 'https://www.sfgate.com/sports/'), ('lastmod', '2021-04-06T15:50:21Z')]"
3,https://www.sfgate.com/sitemap/5000-10000.xml,"[('loc', 'https://www.sfgate.com/rss/'), ('lastmod', '2021-03-31T16:57:41Z')]"
4,https://www.sfgate.com/sitemap/5000-10000.xml,"[('loc', 'https://www.sfgate.com/raiders/'), ('lastmod', '2021-03-31T16:57:41Z')]"
...,...,...
1100959,https://www.sfgate.com/sitemap/16120000-16125000.xml,"[('loc', 'https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php'), ('lastmod', '2021-04-22T15:45:03Z')]"
1100960,https://www.sfgate.com/sitemap/16125000-16130000.xml,"[('loc', 'https://www.sfgate.com/renotahoe/article/2021-04-earthquake-tahoe-truckee-reno-16127479.php'), ('lastmod', '2021-04-25T16:03:24Z')]"
1100961,https://www.sfgate.com/sitemap/16125000-16130000.xml,"[('loc', 'https://www.sfgate.com/california-wildfires/article/2021-04-supertanker-plane-shut-down-calif-fires-16127394.php'), ('lastmod', '2021-04-25T14:42:28Z')]"
1100962,https://www.sfgate.com/sitemap/16125000-16130000.xml,"[('loc', 'https://www.sfgate.com/bayarea/article/2021-04-Fifth-whale-washes-up-dead-in-SF-Bay-16126769.php'), ('lastmod', '2021-04-25T01:44:09Z')]"


In [11]:
df['articles'] = df['articles'].parallel_apply(lambda x: OrderedDict(eval(x)))
df

,page,articles
0,https://www.sfgate.com/sitemap/5000-10000.xml,"{'loc': 'https://www.sfgate.com/', 'lastmod': '2021-04-20T22:15:07Z'}"
1,https://www.sfgate.com/sitemap/5000-10000.xml,"{'loc': 'https://www.sfgate.com/realestate/', 'lastmod': '2021-04-14T20:12:18Z'}"
2,https://www.sfgate.com/sitemap/5000-10000.xml,"{'loc': 'https://www.sfgate.com/sports/', 'lastmod': '2021-04-06T15:50:21Z'}"
3,https://www.sfgate.com/sitemap/5000-10000.xml,"{'loc': 'https://www.sfgate.com/rss/', 'lastmod': '2021-03-31T16:57:41Z'}"
4,https://www.sfgate.com/sitemap/5000-10000.xml,"{'loc': 'https://www.sfgate.com/raiders/', 'lastmod': '2021-03-31T16:57:41Z'}"
...,...,...
1100959,https://www.sfgate.com/sitemap/16120000-16125000.xml,"{'loc': 'https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php', 'lastmod': '2021-04-22T15:45:03Z'}"
1100960,https://www.sfgate.com/sitemap/16125000-16130000.xml,"{'loc': 'https://www.sfgate.com/renotahoe/article/2021-04-earthquake-tahoe-truckee-reno-16127479.php', 'lastmod': '2021-04-25T16:03:24Z'}"
1100961,https://www.sfgate.com/sitemap/16125000-16130000.xml,"{'loc': 'https://www.sfgate.com/california-wildfires/article/2021-04-supertanker-plane-shut-down-calif-fires-16127394.php', 'lastmod': '2021-04-25T14:42:28Z'}"
1100962,https://www.sfgate.com/sitemap/16125000-16130000.xml,"{'loc': 'https://www.sfgate.com/bayarea/article/2021-04-Fifth-whale-washes-up-dead-in-SF-Bay-16126769.php', 'lastmod': '2021-04-25T01:44:09Z'}"


In [42]:
# df = pd.json_normalize(df.articles)
# df.columns= ['url','lastmod','name','language','date','title','image']
# df = df[['url','date','title']]
# df = df[['loc','lastmod']]
# df.columns= ['url', 'date']
# df = df[['url','date','title']]
df = df[['url']]
df

,url
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php
3,https://www.sfgate.com/news/article/It-s-On-Zynga-CEO-Mark-Pincus-Hits-The-Road-Next-2303565.php
4,https://www.sfgate.com/news/article/You-ve-Never-Heard-Of-SoJo-But-Ellen-DeGeneres-2303581.php
...,...
496399,https://www.sfgate.com/news/article/2021-04-california-skydiving-lodi-parachute-death-16121210.php
496400,https://www.sfgate.com/crime/article/2021-04-Kristin-Smart-Paul-Flores-bail-16120845.php
496401,https://www.sfgate.com/bayarea/article/2021-04-video-Tyrell-Wilson-Andrew-Hall-shooting-16120532.php
496402,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php


In [43]:
df.to_csv('sfgate_2.csv', index=False)

In [27]:
df= pd.read_csv('sfgate_2.csv', header=0)
df

,url,date
0,https://www.sfgate.com/,2021-04-20T22:15:07Z
1,https://www.sfgate.com/realestate/,2021-04-14T20:12:18Z
2,https://www.sfgate.com/sports/,2021-04-06T15:50:21Z
3,https://www.sfgate.com/rss/,2021-03-31T16:57:41Z
4,https://www.sfgate.com/raiders/,2021-03-31T16:57:41Z
...,...,...
1100959,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php,2021-04-22T15:45:03Z
1100960,https://www.sfgate.com/renotahoe/article/2021-04-earthquake-tahoe-truckee-reno-16127479.php,2021-04-25T16:03:24Z
1100961,https://www.sfgate.com/california-wildfires/article/2021-04-supertanker-plane-shut-down-calif-fires-16127394.php,2021-04-25T14:42:28Z
1100962,https://www.sfgate.com/bayarea/article/2021-04-Fifth-whale-washes-up-dead-in-SF-Bay-16126769.php,2021-04-25T01:44:09Z


In [31]:
df[df.url.str.contains('/news/|/nation/|/bayarea/|/opinion/|/world/|/politics/|/crime/|/elections/')]
df[df.url.str.contains('/article/')]

,url,date
3158,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php,2020-07-15T05:20:18Z
3159,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php,2020-07-15T02:48:30Z
3160,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php,2020-07-15T02:48:51Z
3161,https://www.sfgate.com/news/article/It-s-On-Zynga-CEO-Mark-Pincus-Hits-The-Road-Next-2303565.php,2020-07-15T02:48:30Z
3162,https://www.sfgate.com/news/article/You-ve-Never-Heard-Of-SoJo-But-Ellen-DeGeneres-2303581.php,2020-07-15T02:48:30Z
...,...,...
1100959,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php,2021-04-22T15:45:03Z
1100960,https://www.sfgate.com/renotahoe/article/2021-04-earthquake-tahoe-truckee-reno-16127479.php,2021-04-25T16:03:24Z
1100961,https://www.sfgate.com/california-wildfires/article/2021-04-supertanker-plane-shut-down-calif-fires-16127394.php,2021-04-25T14:42:28Z
1100962,https://www.sfgate.com/bayarea/article/2021-04-Fifth-whale-washes-up-dead-in-SF-Bay-16126769.php,2021-04-25T01:44:09Z


In [44]:
df = df[df.url.str.contains('/(news|local|nation|bayarea|opinion|world|politics|crime|elections)/')\
        & df.url.str.contains('/article/')].reset_index(drop=True)
df

,url
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php
3,https://www.sfgate.com/news/article/It-s-On-Zynga-CEO-Mark-Pincus-Hits-The-Road-Next-2303565.php
4,https://www.sfgate.com/news/article/You-ve-Never-Heard-Of-SoJo-But-Ellen-DeGeneres-2303581.php
...,...
496399,https://www.sfgate.com/news/article/2021-04-california-skydiving-lodi-parachute-death-16121210.php
496400,https://www.sfgate.com/crime/article/2021-04-Kristin-Smart-Paul-Flores-bail-16120845.php
496401,https://www.sfgate.com/bayarea/article/2021-04-video-Tyrell-Wilson-Andrew-Hall-shooting-16120532.php
496402,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php


#  过滤一下

In [38]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [45]:
df

,url
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php
3,https://www.sfgate.com/news/article/It-s-On-Zynga-CEO-Mark-Pincus-Hits-The-Road-Next-2303565.php
4,https://www.sfgate.com/news/article/You-ve-Never-Heard-Of-SoJo-But-Ellen-DeGeneres-2303581.php
...,...
496399,https://www.sfgate.com/news/article/2021-04-california-skydiving-lodi-parachute-death-16121210.php
496400,https://www.sfgate.com/crime/article/2021-04-Kristin-Smart-Paul-Flores-bail-16120845.php
496401,https://www.sfgate.com/bayarea/article/2021-04-video-Tyrell-Wilson-Andrew-Hall-shooting-16120532.php
496402,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php


In [46]:
from collections import Counter

head = df['url'] 
# + ' ' + df['title']

df['head_in'] = head.parallel_apply(find_relevant)
df['head_out'] = head.parallel_apply(find_exclude)

head_in = df['head_in'].str.len()
head_out = df['head_out'].str.len()

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df[remove_index== True].index
remove_index.shape

/home/yibo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/yibo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


(36374,)

In [48]:
# df.date = pd.to_datetime(df.date)
# df['date'] = df['date'].dt.strftime('%Y/%m/%d')

In [58]:
# df = df[~df.duplicated(subset='url')].reset_index(drop=True)
# df = df[~df.duplicated(subset=['title', 'date'])]
#  df[df.index.isin(remove_index)]
df

,url,head_in,head_out
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php,{},{}
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php,{'kill': 1},{}
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php,{},{}
3,https://www.sfgate.com/news/article/10-Things-You-Need-To-Know-This-Morning-NOK-2303636.php,{},{}
4,https://www.sfgate.com/news/article/Box-net-Forges-Closer-Ties-With-Google-2303673.php,{},{}
...,...,...,...
326737,https://www.sfgate.com/crime/article/2021-04-Oakland-Markus-Mcglothen-arrest-roommate-16121597.php,"{'crime': 1, 'arrest': 1}",{}
326738,https://www.sfgate.com/news/article/2021-04-california-skydiving-lodi-parachute-death-16121210.php,{'death': 1},{}
326739,https://www.sfgate.com/bayarea/article/2021-04-video-Tyrell-Wilson-Andrew-Hall-shooting-16120532.php,{'shooting': 1},{}
326740,https://www.sfgate.com/bayarea/article/2021-04-California-CDC-data-lowest-case-rate-US-16120284.php,{'case': 1},{}


In [55]:
df = df[~df.url.str.contains('E|earn')]

In [56]:
df  = df[~df.index.isin(remove_index)].reset_index(drop=True)

In [66]:
df = df[~df.url.str.contains('201[3-9]-')]
df = df[~df.url.str.contains('202[0-2]-')]
df = df.reset_index(drop=True)
df

,url,head_in,head_out
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php,{},{}
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php,{'kill': 1},{}
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php,{},{}
3,https://www.sfgate.com/news/article/10-Things-You-Need-To-Know-This-Morning-NOK-2303636.php,{},{}
4,https://www.sfgate.com/news/article/Box-net-Forges-Closer-Ties-With-Google-2303673.php,{},{}
...,...,...,...
326165,https://www.sfgate.com/local/article/Bay-Area-vaccine-fairy-has-booked-more-than-16105028.php,{},{}
326166,https://www.sfgate.com/crime/article/Video-Asian-woman-robbed-gunpoint-San-Francisco-16111781.php,{'crime': 1},{}
326167,https://www.sfgate.com/bayarea/article/parklet-car-crash-San-Francisco-Napper-Tandy-16116404.php,{},{}
326168,https://www.sfgate.com/bayarea/article/San-Francisco-mayor-London-Breed-Derek-Chauvin-16117093.php,{'mayor': 1},{}


In [ ]:
df_others = pd.read_csv('sfgate_2013_2021.csv', header=0)
df_others

In [70]:
df = df[~df.url.isin(df_others.url)].reset_index(drop=True)
df

,url,head_in,head_out
0,https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php,{},{}
1,https://www.sfgate.com/news/article/Coursekit-The-Student-Made-Blackboard-Killer-2303165.php,{'kill': 1},{}
2,https://www.sfgate.com/news/article/RIM-Gets-An-Upgrade-It-s-Too-Cheap-To-Be-True-2303520.php,{},{}
3,https://www.sfgate.com/news/article/10-Things-You-Need-To-Know-This-Morning-NOK-2303636.php,{},{}
4,https://www.sfgate.com/news/article/Box-net-Forges-Closer-Ties-With-Google-2303673.php,{},{}
...,...,...,...
321988,https://www.sfgate.com/local/article/Bay-Area-vaccine-fairy-has-booked-more-than-16105028.php,{},{}
321989,https://www.sfgate.com/crime/article/Video-Asian-woman-robbed-gunpoint-San-Francisco-16111781.php,{'crime': 1},{}
321990,https://www.sfgate.com/bayarea/article/parklet-car-crash-San-Francisco-Napper-Tandy-16116404.php,{},{}
321991,https://www.sfgate.com/bayarea/article/San-Francisco-mayor-London-Breed-Derek-Chauvin-16117093.php,{'mayor': 1},{}


In [71]:
df.to_csv('sfgate_3.csv', index=False)

In [73]:
num_chunks = 20
df_save = np.array_split(df, num_chunks)
save_chunks = lambda x: df_save[x].to_csv('sfgate_%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [74]:
del df_save

In [4]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        article.download()
        time.sleep(2)
        article.parse()
        soup = BeautifulSoup(article.html)
        date = soup.find_all('time',class_=re.compile('timestamp.*pubdate'))
        try:
            date = date[0]['datetime']
        except:
            date =None
        
        return pd.Series([article.text,article.publish_date,date])
    except:
        print('error %s'%url)
        return pd.Series([None]*3)

# 正在爬

In [5]:
pandarallel.initialize(nb_workers=8, progress_bar=False)
batch_size= 8*20

for idx in range(20):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('sfgate_%03d.csv'%idx, header=0)
    
    i=0
while i < df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1,['text','date1','date2']]=\
    df.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_story).values
    i += batch_size
#         if i % (batch_size * 10) ==0:
#             df.to_csv('nbc/%03d.csv'%idx, index=False)
df.to_csv('nbc/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------0 ------------
0,error https://www.sfgate.com/news/article/ShoeDazzle-Birchbox-And-Other-Box-At-Your-Door-2303665.php
error https://www.sfgate.com/politics/article/Mitt-Romney-lures-away-100-Obama-2008-donors-2304054.phperror https://www.sfgate.com/news/article/Who-Is-This-Woman-And-How-Is-Her-Startup-Worth-2304723.php

error https://www.sfgate.com/news/article/RUMOR-Next-Xbox-Will-Be-Smaller-Cheaper-And-2339608.php
error https://www.sfgate.com/news/article/Here-Is-Charlie-Cheever-s-Favorite-Quora-Page-2360357.phperror https://www.sfgate.com/news/article/Foursquare-And-Facebook-Places-Are-Still-2361471.phperror https://www.sfgate.com/news/article/Microsoft-Should-Have-Let-Google-Have-Search-To-2363260.php
error https://www.sfgate.com/news/article/Google-Blew-It-And-This-Startup-Seized-The-2362693.php
error https://www.sfgate.com/news

KeyboardInterrupt: 


error https://www.sfgate.com/bayarea/article/Police-plan-OKd-to-avoid-crises-with-mentally-ill-2476454.phperror https://www.sfgate.com/news/article/Blogger-stays-in-prison-defying-grand-jury-order-2468168.phperror https://www.sfgate.com/opinion/fiore/article/Reform-Madness-2479204.phperror https://www.sfgate.com/politics/article/GOP-senator-cites-Dem-rival-s-sexy-novels-2486560.phperror https://www.sfgate.com/news/article/Cover-letters-that-catch-the-eye-2479779.phperror https://www.sfgate.com/nation/article/Feds-put-nation-s-pipeline-operators-on-notice-2479393.phperror https://www.sfgate.com/news/article/Th-ink-2479107.php

































error https://www.sfgate.com/news/article/Bit-ly-Raises-10-Million-Stays-Bullish-On-Link-2478440.php
error https://www.sfgate.com/news/article/Apple-Sold-250-000-Apple-TVs-AAPL-2478073.php
error https://www.sfgate.com/politics/article/Ivory-Coast-president-may-be-removed-by-force-2479760.php
error https://www.sfgate.com/news/articl

In [ ]:
pandarallel.initialize(nb_workers=16, progress_bar=False)
batch_size= 16*20

for idx in range(20):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('sfgate_%03d.csv'%idx, header=0)
    
    i=0
    while i < df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1,['text','date1','date2']]=\
        df.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_story).values
        i += batch_size
#         if i % (batch_size * 10) ==0:
#             df.to_csv('nbc/%03d.csv'%idx, index=False)
    df.to_csv('nbc/%03d.csv'%idx, index=False)

In [ ]:
df_all=[]
for idx in range(20):
#     print('\n ----------%s ------------'%idx)
    df = pd.read_csv('sfgate_%03d.csv'%idx, header=0)
    df_all.append(df)
df_all = pd.concat(df_all).reset_index(drop=True)
df_all